Věrohodnost

In [1]:
import numpy as np
import pandas as pd 


import matplotlib.pyplot as plt
import scipy.optimize as optimize
import scipy
import seaborn as sns
import json
from zipfile import ZipFile

1. 
Weibullovo rozdelenie
$$
f(x) = 
= 
\begin{cases}
\frac{k}{\lambda}(\frac{x}{\lambda}^{k-1})\text{e}^{-(\frac{x}{\lambda})^k} & \text{ if } x \geq 0.5\\
0 & \text{ if } x < 0.5
\end{cases}
$$

$$ 
k \in (0, \infty) 
$$ 
k je shape,tvar parameter
$$ 
\lambda \in (0, \infty) 
$$ 
λ je scale, meritko parameter

#### Logaritmická-verohodnostna funkcia
$$
l(k, \lambda) = n*\ln{k} - n*k*\ln{\lambda} + (k+1) * \sum_{i=1}^{n} \ln{x_{i}} - \sum_{i=1}^{n} (\frac{x_{i}}{\lambda})^k
$$

#### Parciálna derivácia podľa parametru shape
$$
\frac{\partial l}{\partial k} = \frac{n}{k} - n*\ln{\lambda} + \sum_{i=1}^{n} \ln{x_{i}} - \sum_{i=1}^{n} (\ln{x_{i}} - \ln{\lambda}) * (\frac{x_{i}}{\lambda})^k
$$
#### Parciálna derivácia podľa parametru scale
$$
\frac{\partial l}{\partial \lambda} = - \frac{n*k}{\lambda} + \frac{k}{\lambda^{k+1}} * \sum_{i=1}^{n} x_{i}^{k}
$$

In [2]:
# load data
fileName = "Data_2024.xlsx"
df = pd.read_excel(fileName, sheet_name="Data_věrohodnost") # need pip install xlrd and openpyxl
df = df.iloc[:, :-2] # remove columns without information
# check types
df['censored'] = df['censored'].astype(int)
df['doba práce v oboru [roky]'] = df['doba práce v oboru [roky]'].astype(float)
print(df)

     censored  doba práce v oboru [roky]
0           0                      6.528
1           0                      6.013
2           1                      6.055
3           0                      7.243
4           1                      5.629
..        ...                        ...
316         1                      5.562
317         0                      5.491
318         0                      6.761
319         0                      7.062
320         0                      5.784

[321 rows x 2 columns]


In [78]:
x = df['doba práce v oboru [roky]'].values
censored = df['censored'].values

# logaritmicka-verohodnostna funkcia
def log_likelihood(params):
    k, lamb = params
    # log density - log of pdf
    log_pdf = np.log(k) - np.log(lamb) * k + (k-1) * np.log(x) - ((x/lamb) ** k)
    sf = np.exp(-(x / lamb)**k)  # survivor function - from CDF
    # Logaritmická věrohodnost
    log_l = (1 - censored) * log_pdf + censored * np.log(sf)
    return -np.sum(log_l)  # negation for minimization

# Počáteční odhady
initial_params = [1.0, 1.0]

# Optimalizace
result = optimize.minimize(log_likelihood, initial_params, method='L-BFGS-B')

# Výsledky
if result.success:
    k_mle, lam_mle = result.x
    print(f"Odhadnuté parametry: k = {k_mle:.4f}, λ = {lam_mle:.4f}")
else:
    print("Optimalizace selhala:", result.message)

Odhadnuté parametry: k = 6.1728, λ = 7.4295


$$\hat k = 6.1728 $$

$$\hat λ = 7.4295$$

3) Test verohodnostnym pomerom

In [ ]:
# fix k to 1 and estimate MLE again
x = df['doba práce v oboru [roky]'].values
censored = df['censored'].values

# logaritmicka-verohodnostna funkcia
def log_likelihood(k_fix, lamb):
    k = k_fix
    # log density - log of pdf
    log_pdf = np.log(k) - np.log(lamb) * k + (k-1) * np.log(x) - ((x/lamb) ** k)
    sf = np.exp(-(x / lamb)**k)  # survivor function - from CDF
    # Logaritmická věrohodnost
    log_l = (1 - censored) * log_pdf + censored * np.log(sf)
    return -np.sum(log_l)  # negation for minimization

# Počáteční odhady
initial_lamb = [1.0]
k_fix = 1.0

# Optimalizace
result_exp = optimize.minimize(lambda lamb:log_likelihood(k_fix, lamb), initial_lamb, method='L-BFGS-B', bounds=[(1e-5, None)])

# Výsledky
if result_exp.success:
    lam_exp_mle = result_exp.x[0]
    print(f"Odhadnutý parameter: λ = {lam_exp_mle:.4f}")
else:
    print("Optimalizace selhala:", result_exp.message)

print(result_exp.fun)

Odhadnutý parameter: λ = 9.0533
746.3288140610031


In [87]:
lr = -2*(-result_exp.fun - (-result.fun))
print(lr)

592.3898153434229


Test
$$
H_0: k = 1 \text{Stačí exponenciálne rozdelenia.}
$$
$$
H_A: k \neq 1 \text{Exponenciálne rozdelenia nestačí.}
$$
$$
stupne volnosti 2 - 1 = 1
W'_0.05 = <0,\Chi^2 0.95(1)> = <0, 3.841>
$$
testova statistika = 592.39 nelezi v kritickom obore, preto $H_0$ zamietam

4. Bodové odhady pre strednu dobu zamestania v obore 
10% percentil zamestnania v obore

In [102]:
import scipy
# mean of weibull random variable is mean time in the field
Ex = lam_mle * scipy.special.factorial((1 + 1/k_mle) - 1)
print(f"Stredna doba zamestnania v obore je {Ex:.4f} rokov.")

# 10% employment in the field
p = 0.1
percentile = lam_mle * (-np.log(1-p))**(1/k_mle)
print(f"10% percentil zamestnania v obore je {percentile:.4f} rokov.")

Stredna doba zamestnania v obore je 6.9032 rokov.
10% percentil zamestnania v obore je 5.1598 rokov.


Odpoveď: 
Stredna doba zamestnania v obore je 6.9032 rokov.
10% percentil zamestnania v obore je 5.1598 rokov.

Väčšina ľudí pracuje obore do 7 rokov, desať percent ľudí zmení obor po piatich rokoch. 

Regrese

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.graphics.gofplots import qqplot
from matplotlib import cm

In [ ]:
# načítaj dáta z xlsx súboru
fileName = "Data_2024.xlsx"
df = pd.read_excel(fileName, sheet_name="Data_regrese") # need pip install xlrd and openpyxl
# skontroluj typy
df['ActiveUsers'] = df['ActiveUsers'].astype(int)
df['InteractingPct'] = df['InteractingPct'].astype(float)
df['ScrollingPct'] = df['ScrollingPct'].astype(float)
df['Ping [ms]'] = df['Ping [ms]'].astype(float)
print(df)

In [ ]:
print(df["OSType"].unique()) # ktoré OS sú v dátach

In [ ]:
# nahrad OSType stlpec s one-hot encoding ekvivalentom
df = pd.concat([df.drop(columns=["OSType", ]), pd.get_dummies(df["OSType"])], axis=1)
#  premenuj ping aby sa dal pouzit v ols formuly
df = pd.concat([df.drop(columns=["Ping [ms]"]), df["Ping [ms]"].rename("ping")], axis=1)
print(df)

Plný kvadratický model je:
$$
\displaystyle
\begin{align*}
\text{Ping} &= \beta_0 + \beta_1 \text{ActiveUsers} + \beta_2 \text{InteractingPct} + \beta_3 \text{ScrollingPct} \\
&+ \beta_4 \text{iOS} + \beta_5 \text{Windows} + \beta_6 \text{Android}\\
&+ \beta_{11} (\text{ActiveUsers}^2) + \beta_{22} (\text{InteractingPct}^2) + \beta_{33} (\text{ScrollingPct}^2) \\
&+ \beta_{12} ( \text{ActiveUsers} \times \text{InteractingPct}) + \beta_{13} ( \text{ActiveUsers} \times \text{ScrollingPct}) \\
&+ \beta_{23} ( \text{InteractingPct} \times \text{ScrollingPct}) \\
&+ \beta_{14} ( \text{ActiveUsers} \times \text{iOS}) + \beta_{15} ( \text{ActiveUsers} \times \text{Windows}) + \beta_{16} ( \text{ActiveUsers} \times \text{Android}) \\
&+ \beta_{24} ( \text{InteractingPct} \times \text{iOS}) + \beta_{25} ( \text{InteractingPct} \times \text{Windows}) + \beta_{26} ( \text{InteractingPct} \times \text{Android}) \\
&+ \beta_{34} ( \text{ScrollingPct} \times \text{iOS}) + \beta_{35} ( \text{ScrollingPct} \times \text{Windows}) + \beta_{36} ( \text{ScrollingPct} \times \text{Android})
+ \epsilon
\end{align*}
$$

In [ ]:
# definuj funkciu pre zostavenie modelu a zobrazenie summary, VIF a korelacie prediktorov
def buildModelRunStats(formula, dframe):
    model=smf.ols(formula=formula,data=dframe) # přepiš původní (lineární) model
    results=model.fit()
    print(results.summary())
    #upozorni na významnost koeficientů, R^2 atp
    # získej VIF a slož všechny VIF do dataframe
    X = pd.DataFrame(model.exog, columns=model.exog_names)
    vif = pd.Series([variance_inflation_factor(X.values, i)
                    for i in range(X.shape[1])],
                    index=X.columns)
    vif_df = vif.to_frame()
    # Nastavení názvu sloupce
    vif_df.columns = ['VIF']
    print('\n\n\n')
    print("VIF")
    print(vif_df)
    #ukaž korelaci prediktorů
    print('\n\n\n')
    print("Compute pairwise correlation of columns")
    print(X.corr())

def buildModelSummary(formula, dframe):
    model=smf.ols(formula=formula,data=dframe) # přepiš původní (lineární) model
    results=model.fit()
    print(results.summary())
    return results

def buildModelPrintVIF(formula, dframe):
    model=smf.ols(formula=formula,data=dframe) # přepiš původní (lineární) model
    #upozorni na významnost koeficientů, R^2 atp
    # získej VIF a slož všechny VIF do dataframe
    X = pd.DataFrame(model.exog, columns=model.exog_names)
    vif = pd.Series([variance_inflation_factor(X.values, i)
                    for i in range(X.shape[1])],
                    index=X.columns)
    vif_df = vif.to_frame()
    # Nastavení názvu sloupce
    vif_df.columns = ['VIF']
    print("VIF")
    print(vif_df)

In [ ]:
formula = 'ping~ActiveUsers + InteractingPct + ScrollingPct + iOS + Windows + Android + I(ActiveUsers**2) + I(InteractingPct**2) + I(ScrollingPct**2) + ActiveUsers:InteractingPct + ActiveUsers:ScrollingPct + InteractingPct:ScrollingPct + ActiveUsers:iOS + ActiveUsers:Windows + ActiveUsers:Android + InteractingPct:iOS + InteractingPct:Windows + InteractingPct:Android + ScrollingPct:iOS + ScrollingPct:Windows + ScrollingPct:Android'
buildModelRunStats(formula, df)

In [ ]:
buildModelPrintVIF(formula, df)

In [ ]:
# overenie korelacie
correlationOut = df[["InteractingPct", "ScrollingPct"]].corr()
print(correlationOut)

Podľa riadku: InteractingPct:ScrollingPct                     1.000000

existuje perfektná korelácia medzi závislými premennými InteractingPct a ScrollingPct, preto použijem len jednu. Ďalej som zvolil, že budem používať InteractingPct.

In [ ]:
# formula bez ScrollingPct
formula = 'ping~ActiveUsers + InteractingPct + iOS + Windows + Android + I(ActiveUsers**2) + I(InteractingPct**2) + ActiveUsers:InteractingPct + ActiveUsers:iOS + ActiveUsers:Windows + ActiveUsers:Android + InteractingPct:iOS + InteractingPct:Windows + InteractingPct:Android'
buildModelRunStats(formula, df)

In [ ]:
# bez ScrollingPct
buildModelPrintVIF(formula, df)

Existuje korelácia medzi ActiveUsers a InteractingPct
použijem z-scores $\frac{x-\bar{x}}{s(x)}$ štandardizácia na 0 priemer a 1 smerodatnú odchylku 

In [ ]:
dfSt=df.copy()
# odcitaj priemer a podel smerodatnou odchylkou
dfSt['ActiveUsers']=(dfSt['ActiveUsers']-dfSt['ActiveUsers'].mean())/dfSt['ActiveUsers'].std()
dfSt['InteractingPct']=(dfSt['InteractingPct']-dfSt['InteractingPct'].mean())/dfSt['InteractingPct'].std()

In [ ]:
# so standardizaciou ActiveUsers a InteractingPct
formula = 'ping~ActiveUsers + InteractingPct + iOS + Windows + Android + I(ActiveUsers**2) + I(InteractingPct**2) + ActiveUsers:InteractingPct + ActiveUsers:iOS + ActiveUsers:Windows + ActiveUsers:Android + InteractingPct:iOS + InteractingPct:Windows + InteractingPct:Android'
buildModelRunStats(formula, dfSt)

In [ ]:
# so standardizaciou ActiveUsers a InteractingPct
buildModelPrintVIF(formula, dfSt)

Teraz je VIF akceptovateľný (VIF < 10).
Môžem začať so spätnou elimináciou, eliminujem kým nedosiahnem p < 0.05 pre všetky interakcie.

In [ ]:
# este raz spustim so standardizaciou ActiveUsers a InteractingPct
formula = 'ping~ActiveUsers + InteractingPct + iOS + Windows + Android + I(ActiveUsers**2) + I(InteractingPct**2) + ActiveUsers:InteractingPct + ActiveUsers:iOS + ActiveUsers:Windows + ActiveUsers:Android + InteractingPct:iOS + InteractingPct:Windows + InteractingPct:Android'
buildModelPrintVIF(formula, dfSt)

Najväčšia hodnota je 0.888 pre InteractingPct:Android, preto tento člen vymažem

In [ ]:
# vymazany InteractingPct:Android
formula = 'ping~ActiveUsers + InteractingPct + iOS + Windows + Android + I(ActiveUsers**2) + I(InteractingPct**2) + ActiveUsers:InteractingPct + ActiveUsers:iOS + ActiveUsers:Windows + ActiveUsers:Android + InteractingPct:iOS + InteractingPct:Windows'
buildModelPrintVIF(formula, dfSt)

Ďalší bol člen InteractingPct:iOS 0.828

In [ ]:
# vymazany InteractingPct:iOS
formula = 'ping~ActiveUsers + InteractingPct + iOS + Windows + Android + I(ActiveUsers**2) + I(InteractingPct**2) + ActiveUsers:InteractingPct + ActiveUsers:iOS + ActiveUsers:Windows + ActiveUsers:Android + InteractingPct:Windows'
buildModelPrintVIF(formula, dfSt)

InteractingPct:Windows 0.821 

In [ ]:
# bez InteractingPct:Windows
formula = 'ping~ActiveUsers + InteractingPct + iOS + Windows + Android + I(ActiveUsers**2) + I(InteractingPct**2) + ActiveUsers:InteractingPct + ActiveUsers:iOS + ActiveUsers:Windows + ActiveUsers:Android'
buildModelPrintVIF(formula, dfSt)

I(InteractingPct ** 2) 0.278 

In [ ]:
# bez I(InteractingPct ** 2)
formula = 'ping~ActiveUsers + InteractingPct + iOS + Windows + Android + I(ActiveUsers**2) + ActiveUsers:InteractingPct + ActiveUsers:iOS + ActiveUsers:Windows + ActiveUsers:Android'
resModel = buildModelPrintVIF(formula, dfSt)

Už neostali žiadne členy s p-hodnotou >= 0.05.

#### 1 a) Rovnica výsledného modelu

$$
\displaystyle
\begin{align*}
\text{Ping} &= \beta_0 + \beta_1 \text{ActiveUsers} + \beta_2 \text{InteractingPct} \\
&+ \beta_4 \text{iOS} + \beta_5 \text{Windows} + \beta_6 \text{Android}\\
&+ \beta_{11} (\text{ActiveUsers}^2) \\
&+ \beta_{12} ( \text{ActiveUsers} \times \text{InteractingPct})  \\
&+ \beta_{14} ( \text{ActiveUsers} \times \text{iOS}) + \beta_{15} ( \text{ActiveUsers} \times \text{Windows}) \\
&+ \beta_{16} ( \text{ActiveUsers} \times \text{Android}) + \epsilon
\end{align*}
$$

##### Kontrola či dáta neobsahujú outliery.
Najskôr si zobrazím graf reziduí a potom spočítam Cookovu vzdialenosť.

In [ ]:
# graf rezidui
# res vs ping
plt.scatter(dfSt['ping'], resModel.resid)
plt.axhline(y=0, color='r', linestyle='-')
plt.grid(True)
plt.title("Rezidua vs Ping")
plt.xlabel("Ping")
plt.ylabel("Rezidua")

plt.show()

In [ ]:
influence = resModel.get_influence()
# Leverage
leverage = influence.hat_matrix_diag
# Cookovy D hodnoty (a p-hodnoty) jako n-tice polí [n x 2]
cooks_d = influence.cooks_distance
# Standardizovaná rezidua
standardized_residuals = influence.resid_studentized_internal
# Studentizovaná rezidua
studentized_residuals = influence.resid_studentized_external

#tabulka dohromady dořešit změny počtu řádků
outl_stats_df = pd.DataFrame({
    'Leverage': leverage,
    'Standardized Residuals': standardized_residuals,
    'Studentized Residuals': studentized_residuals,
    'Cook\'s Distance': cooks_d[0],
    'Cook\'s Distance_p-value': cooks_d[1]
}, index=dfSt.index)
#vyber jen "zajímavý" hodnoty
outl_stats_df = outl_stats_df[(outl_stats_df['Leverage'] > 3*len(resModel.params)/dfSt.shape[0]) | (np.abs(outl_stats_df['Standardized Residuals']) > 2) | (outl_stats_df['Cook\'s Distance_p-value'] < 0.05)]


print(outl_stats_df)

Podľa grafu reziduí je vidieť, že sa dve hodnoty veľmi líšia od ostatných - sú to hľadané outliery. Podľa výpočtu štandardizovaných a študentizovaných reziduí sú najviac líšiace sa merania s indexmi 255 a 476. Preto ich v ďalšom kroku odstránim a znovu vizualizujem graf reziduí.

In [ ]:
# vymazanie outlierov
outlierIndxs = [255,476]
print(f"Drop: {outlierIndxs}")
dfStNoOut = dfSt.drop(index=outlierIndxs)

In [ ]:
# model bez outlierov
formula = 'ping~ActiveUsers + InteractingPct + iOS + Windows + Android + I(ActiveUsers**2) + ActiveUsers:InteractingPct + ActiveUsers:iOS + ActiveUsers:Windows + ActiveUsers:Android'
finalModel = buildModelSummary(formula, dfStNoOut)

In [ ]:
# graf rezidui
# res vs ping
plt.scatter(dfStNoOut['ping'], finalModel.resid)
plt.axhline(y=0, color='r', linestyle='-')
plt.grid(True)
plt.title("Rezidua vs Ping")
plt.xlabel("Ping")
plt.ylabel("Rezidua")

plt.show()

Odstránil som správne dve hodnoty a model sa podľa koeficientu determinácie zlepšil z 0.843 na 0.877. Odstránené hodnoty mohli byť valídne avšak bolo by potreba viac dát s podobnými hodnotami aby som ich mohol použiť.

Teraz je model hotový.